# 🚀 Proyecto ELT con SpaceX API  
## Notebook 02 – Transformación de datos

En este notebook se aplican tareas de **procesamiento y transformación de datos con Pandas**, según la consigna:  

- Conversión de tipos. 
- Eliminación de duplicados.  
- Manejo de valores nulos. 
- Eliminación de columnas. 
- Renombrado de columnas.  

In [ ]:
import sys

# Ajusto para que la reconozca la ruta del proyecto
sys.path.append("..") 

# --- IMPORTS ---
from src.transform import load_from_parquet, drop_duplicates, clean_dataframe, rename_columns, normalize_columns, drop_columns
from src.load import save_to_parquet 
from src.config import setup_logger

logger = setup_logger()

# --- LOGGER ---
logger = setup_logger()
logger.info("Inicio de transformaciones de datos")


2025-09-06 18:54:40,583 - INFO - Inicio de transformaciones de datos


## 📥 Lectura de datos

Se extraen los datasets desde Delta Lake.  
Trabajaremos principalmente con:  
- **Próximos lanzamientos** (`upcoming_launches`)  
- **Cohetes** (`rockets`)  


In [2]:
# Rockets
# --- Extracción de datos para transformar --- Cargar datos desde Bronze
logger.info("Cargando Rockets desde Bronze...")
rockets_df = load_from_parquet("rockets", layer="bronze")

logger.info("✅ Datos cargados desde Parquet. Inicio de transformaciones de datos.")

2025-09-06 18:54:40,602 - INFO - Cargando Rockets desde Bronze...
2025-09-06 18:54:40,731 - INFO - ✅ Datos cargados desde Parquet. Inicio de transformaciones de datos.


In [3]:
# Upcoming Launches
# --- Extracción de datos para transformar --- Cargar datos desde Bronze
logger.info("Cargando Upcoming Launches desde Bronze...")
upcoming_launches_df = load_from_parquet("upcoming_launches", layer="bronze")

logger.info("✅ Datos cargados desde Parquet. Inicio de transformaciones de datos.")

2025-09-06 18:54:40,750 - INFO - Cargando Upcoming Launches desde Bronze...
2025-09-06 18:54:40,765 - INFO - ✅ Datos cargados desde Parquet. Inicio de transformaciones de datos.


## 🛠 Transformaciones aplicadas en datos Rocket
0. **Aseguramos tipos de datos**.
1. **Eliminar duplicados**.  
2. **Rellenar valores nulos** en la columna `details`.
3. **Elimino columnas**  
4. **Renombrar columnas** para mayor claridad.  


In [4]:
# Normalizo para evitar problemas de incompatibilidad con arrays
rockets_df_normalized = normalize_columns(rockets_df)

# Aplicar transformaciones
logger.info("Aplicando transformaciones en Rockets...")
rockets_df_normalized = drop_duplicates(rockets_df_normalized)
rockets_df_normalized = clean_dataframe(rockets_df_normalized)
rockets_df_normalized = drop_columns(rockets_df_normalized, df_type="rockets")
rockets_df_normalized = rename_columns(rockets_df_normalized, "rockets")

display(rockets_df_normalized.head())

logger.info("✅ Transformaciones aplicadas correctamente.")

2025-09-06 18:54:40,812 - INFO - Aplicando transformaciones en Rockets...


,Nombre,Estado_Activo,Impulsores,Costo_por_Lanzamiento,Tasa_de_Suceso,Fecha_del_Primer_Vuelo,País_de_Fabricación,wikipedia,description,id,...,engines.thrust_vacuum.lbf,Numero_de_Motores,Tipo_de_Motor,Version_de_Motor,Disposición_de_Motor,Pérdida_Máxima_de_Motor,Propelente_Secundario,Relación_Impulso_Peso,Número_de_Patas_de_Aterrizaje,Material_de_Patas_de_Aterrizaje
0,Falcon 1,False,0,6700000,40,2006-03-24,Republic of the Marshall Islands,https://en.wikipedia.org/wiki/Falcon_1,The Falcon 1 was an expendable launch system p...,5e9d0d95eda69955f709d1eb,...,110000,1,merlin,1C,single,0.0,RP-1 kerosene,96.0,0,None
1,Falcon 9,True,0,50000000,98,2010-06-04,United States,https://en.wikipedia.org/wiki/Falcon_9,Falcon 9 is a two-stage rocket designed and ma...,5e9d0d95eda69973a809d1ec,...,205500,9,merlin,1D+,octaweb,2.0,RP-1 kerosene,180.1,4,carbon fiber
2,Falcon Heavy,True,2,90000000,100,2018-02-06,United States,https://en.wikipedia.org/wiki/Falcon_Heavy,With the ability to lift into orbit over 54 me...,5e9d0d95eda69974db09d1ed,...,205500,27,merlin,1D+,octaweb,6.0,RP-1 kerosene,180.1,12,carbon fiber
3,Starship,False,0,7000000,0,2021-12-01,United States,https://en.wikipedia.org/wiki/SpaceX_Starship,Starship and Super Heavy Rocket represent a fu...,5e9d0d96eda699382d09d1ee,...,440000,37,raptor,,None,NaN,liquid methane,107.0,6,stainless steel


2025-09-06 18:54:40,995 - INFO - ✅ Transformaciones aplicadas correctamente.


## 🛠 Transformaciones aplicadas en datos Upcoming Launches
0. **Aseguramos tipos de datos**.
1. **Eliminar duplicados**.  
2. **Elimino columnas**  
3. **Renombrar columnas** para mayor claridad.  


In [5]:
# Normalizo para evitar problemas de incompatibilidad con arrays
upcoming_launches_df_normalized = normalize_columns(upcoming_launches_df)

# Aplicar transformaciones
logger.info("Aplicando transformaciones en Upcoming Launches...")
upcoming_launches_df_normalized = drop_duplicates(upcoming_launches_df_normalized)
upcoming_launches_df_normalized = clean_dataframe(upcoming_launches_df_normalized)
upcoming_launches_df_normalized = drop_columns(upcoming_launches_df_normalized, df_type="upcoming_launches")
upcoming_launches_df_normalized = rename_columns(upcoming_launches_df_normalized, "upcoming_launches")

display(upcoming_launches_df_normalized.head())

logger.info("✅ Transformaciones aplicadas correctamente.")

2025-09-06 18:54:41,038 - INFO - Aplicando transformaciones en Upcoming Launches...


,ID_Cohete,launchpad,Número_de_Vuelo,Nombre,Fecha_Lanzamiento_UTC,Fecha_Lanzamiento_Unix,Fecha_Lanzamiento_Local,Precisión_de_Fecha,cores,id
0,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,188,USSF-44,2022-11-01T13:41:00.000Z,1667310060,2022-11-01T09:41:00-04:00,hour,"[{'core': '5fe3b8f2b3467846b3242181', 'flight'...",6243aec2af52800c6e91925d
1,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,188,Starlink 4-36 (v1.5),2022-10-20T14:50:00.000Z,1666277400,2022-10-20T10:50:00-04:00,hour,"[{'core': None, 'flight': None, 'gridfins': No...",63161345ffc78f3b8567070d
2,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,188,Galaxy 33 (15R) & 34 (12R),2022-10-08T23:05:00.000Z,1665270300,2022-10-08T19:05:00-04:00,hour,"[{'core': None, 'flight': None, 'gridfins': Tr...",633f71240531f07b4fdf59bb
3,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,188,Hotbird 13F,2022-10-15T05:22:00.000Z,1665811320,2022-10-15T01:22:00-04:00,hour,"[{'core': None, 'flight': None, 'gridfins': Tr...",633f71370531f07b4fdf59bc
4,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,189,Hotbird 13G,2022-11-03T03:24:00.000Z,1667445840,2022-11-02T23:24:00-04:00,hour,"[{'core': None, 'flight': None, 'gridfins': Tr...",633f71dd0531f07b4fdf59c1


2025-09-06 18:54:41,098 - INFO - ✅ Transformaciones aplicadas correctamente.


## 💾 Guardado de datos normalizados y transformados en Silver

In [6]:
# Guardar en Silver
logger.info("Guardando transformaciones en Silver...")
save_to_parquet(rockets_df, "rockets", layer="silver", incremental=False, mode="overwrite")

print(f"✅ Transformaciones guardadas en Silver: {len(rockets_df)} filas")

2025-09-06 18:54:41,179 - INFO - Guardando transformaciones en Silver...


[INFO] Guardado FULL en C:\Users\MONSO\OneDrive\Escritorio\Final-DataEngineering\data\silver\rockets\rockets.parquet
✅ Transformaciones guardadas en Silver: 4 filas
